In [1]:
import sys
# The path you want to add  
path_to_add = r'/home/etaylor/code_projects/dubby'  
# Add the directory to sys.path  
sys.path.insert(0, path_to_add)
from src.langchain_utils import initialize_model
from src.prompts import generation_prompts as gp
import pandas as pd

C:\Users\elorberb\AppData\Local\Temp\ipykernel_24932\4189502703.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv(r"C:\Users\elorberb\PycharmProjects\BGU projects\Advanced_NLP\DeltaPredictor\data\updated_data\test_comment.csv")
df.head()

,text,label
0,"comment: Look at the definition you provided, ...",1
1,comment: ∆. Yours was the first comment I rea...,1
2,comment: Maybe a new word is needed? Making su...,1
3,comment: You're using natural to mean definiti...,0
4,comment: Making a political statement is not t...,1


In [6]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import re
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [5]:
filtered_df = df[df['label'] == 0]

# Select the first 25 records from the filtered DataFrame
exp_df = filtered_df.head(25)
exp_df.head()

,text,label
3,comment: You're using natural to mean definiti...,0
10,comment: A lot (dunno %) vegetarians/vegans se...,0
11,"comment: sure, but as i've mentioned several t...",0
12,comment: Another strong pro-veg point is that ...,0
13,comment: It is bad analogy and derails discuss...,0


In [ ]:
def generate_comments(chain, df, n):
    df_copy = df.copy()
    df_copy['generated_comment'] = None

    for i, row in tqdm(df.head(n).iterrows(), total=n):
        comment = row['text']
        input_vars_values = {'comment': comment}  # Assuming 'comment' is defined in your prompt template

        # Generate the response using the language model
        resp = chain(input_vars_values)
        generated_comment = resp['text']  # Adjust based on the actual key in the response
        df_copy.at[i, 'generated_comment'] = generated_comment

    return df_copy


def run_comment_generation(df, azure_deployment_name, num_records, template_dict, input_vars):
    # Initialize the model with the specified parameters
    chain = initialize_model(azure_deployment_name, template_dict['template_text'], input_vars)
    print("Model initialized.")

    # Generate comments for the specified number of records
    df_copy = generate_comments(chain, df, num_records)

    # Here you can add any post-generation processing, such as saving the results or analyzing the generated comments
    return df_copy